<a href="https://colab.research.google.com/github/alimomennasab/ChestXRay-Classification/blob/main/VGGNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import pandas as pd
import os
import numpy as np
from PIL import Image
from glob import glob
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##2. Loading Data

In [ ]:
data_dir = '/content/drive/My Drive/chest_xray/'

In [ ]:
# Split dataset into training, validation, and test sets
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [ ]:
print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['train', 'chest_xray', 'val', '__MACOSX', 'test']
['PNEUMONIA', 'NORMAL']


In [ ]:
# Pneumonia images
pneumonia_files = os.listdir(data_dir + "/train/PNEUMONIA")
print('No. of training examples for Pneumonia:', len(pneumonia_files))
print(pneumonia_files[:5])

No. of training examples for Pneumonia: 3875
['person556_virus_1096.jpeg', 'person536_bacteria_2257.jpeg', 'person581_bacteria_2390.jpeg', 'person592_bacteria_2434.jpeg', 'person581_virus_1125.jpeg']


In [ ]:
# Normal (healthy) images
normal_files = os.listdir(data_dir + "/train/NORMAL")
print('No. of training examples for Normal:', len(normal_files))
print(normal_files[:5])

No. of training examples for Normal: 1341
['IM-0526-0001.jpeg', 'IM-0524-0001.jpeg', 'IM-0507-0001.jpeg', 'IM-0508-0001.jpeg', 'IM-0520-0001.jpeg']


In [ ]:
# There are almost three times more pneumonia images than normal images, so we will use class weighing

# Define classes
classes = ['NORMAL', 'PNEUMONIA']

# Define class weights
num_pneumonia_train = len(os.listdir(os.path.join(train_dir, classes[1])))
num_normal_train = len(os.listdir(os.path.join(train_dir, classes[0])))
total_train = num_pneumonia_train + num_normal_train
class_weights = torch.tensor([total_train/num_normal_train, total_train/num_pneumonia_train]).to(device)

##3. Preparing Dataset and DataLoader

In [ ]:
# Define transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.05)),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

val_and_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

In [ ]:
# Define data directories. We won't use a custom class because the dataset is already well-formatted.

train_dataset = ImageFolder('/content/drive/My Drive/chest_xray/train', transform = train_transform)
val_dataset = ImageFolder('/content/drive/My Drive/chest_xray/val', transform = val_and_test_transform)
test_dataset = ImageFolder('/content/drive/My Drive/chest_xray/test', transform = val_and_test_transform)

train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 16, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = False)

##4. Defining Model

In [ ]:
VGG_types = {
'VGG11' : [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
'VGG13' : [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
'VGG16' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
'VGG19' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}

# Then flatten and 4096 x 4096 x 1000 Linear Layers

In [ ]:
class VGG_net(nn.Module):
    def __init__(self, in_channels = 3, num_classes = 2):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types['VGG19']) # change VGG type here

        self.fcs = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
            x = self.conv_layers(x)
            x = x.reshape(x.shape[0], -1)
            x = self.fcs(x)
            return x

    def create_conv_layers(self, architecture):
            layers = []
            in_channels = self.in_channels

            for x in architecture:
                if type(x) == int:
                    out_channels = x

                    layers += [nn.Conv2d(in_channels=in_channels, out_channels = out_channels,
                                         kernel_size=(3,3), stride=(1,1), padding= (1,1)),
                                         nn.BatchNorm2d(x),
                                         nn.ReLU()]
                    in_channels = x
                elif x == 'M':
                    layers += [nn.MaxPool2d(kernel_size=(2,2), stride= (2,2))]

            return nn.Sequential(*layers)

            return nn.Sequential(*layers)

In [ ]:
# Initializing model
model = VGG_net(in_channels=3, num_classes=2).to(device)

In [ ]:
if torch.cuda.is_available():
    model.cuda()

##5. Testing Model

In [ ]:
x = torch.randn(1, 3, 224, 224)
x = x.to(device) 
print(model(x).shape)

torch.Size([1, 2])


##6. Defining Main Training Loop

In [ ]:
# Hyper-parameters
num_epochs = 30
learning_rate = 0.001
patience = 10

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

curr_lr = learning_rate
total_step = len(train_loader)

# Early stopping parameters
early_stopping_counter = 0
best_loss = float('inf')

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    
    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

    # Calculate validation loss
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels)
        val_loss /= len(val_loader)

        # Save the model if the validation loss is the best observed yet
        if val_loss < best_loss:
            print(f'Saving model with validation loss of {val_loss:.4f}...')
            torch.save(model.state_dict(), 'best_model.pth')
            best_loss = val_loss

    # Early stopping if overfitting
    if early_stopping_counter >= patience:
        print(f'Validation loss has not improved for {patience} epochs. Early stopping...')
        break
    elif val_loss < best_loss:
        best_loss = val_loss
        early_stopping_counter = 0


Epoch [1/30], Step [100/326] Loss: 0.7106
Epoch [1/30], Step [200/326] Loss: 0.7117
Epoch [1/30], Step [300/326] Loss: 0.6990
Saving model with validation loss of 0.6644...
Epoch [2/30], Step [100/326] Loss: 0.6917
Epoch [2/30], Step [200/326] Loss: 0.6744
Epoch [2/30], Step [300/326] Loss: 0.6996
Epoch [3/30], Step [100/326] Loss: 0.6982
Epoch [3/30], Step [200/326] Loss: 0.6797
Epoch [3/30], Step [300/326] Loss: 0.6975
Epoch [4/30], Step [100/326] Loss: 0.6969
Epoch [4/30], Step [200/326] Loss: 0.6872
Epoch [4/30], Step [300/326] Loss: 0.6967
Epoch [5/30], Step [100/326] Loss: 0.6650
Epoch [5/30], Step [200/326] Loss: 0.6837
Epoch [5/30], Step [300/326] Loss: 0.7110
Epoch [6/30], Step [100/326] Loss: 0.7031
Epoch [6/30], Step [200/326] Loss: 0.6994
Epoch [6/30], Step [300/326] Loss: 0.6731
Epoch [7/30], Step [100/326] Loss: 0.6857
Epoch [7/30], Step [200/326] Loss: 0.7042
Epoch [7/30], Step [300/326] Loss: 0.6753
Epoch [8/30], Step [100/326] Loss: 0.7005
Epoch [8/30], Step [200/326] 

##7. Testing

In [ ]:
# Load the saved model checkpoint
checkpoint = torch.load('best_model.pth')
model.load_state_dict(checkpoint)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Model accuracy on test images: {} %'.format(100 * correct / total))

Model accuracy on test images: 62.82051282051282 %
